# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database ERD

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database `data.sqlite`.

In [1]:
# Your code here; import the necessary packages
import pandas as pd
import sqlite3
conn = sqlite3.connect("data.sqlite")

In [2]:
# Your code here; create the connection
# Oops did it above

## Write an Equivalent Query using a Subquery

The following query works using a `JOIN`. Rewrite it so that it uses a subquery instead.

```
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders 
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;
```

In [5]:
# Your code here
pd.read_sql("""
    select customers.customerNumber, customers.contactLastName, customers.contactFirstName
    from customers where customers.customerNumber 
    in (
        select customerNumber from orders
        where orderDate = '2003-01-31'
    )
    

""", conn)

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


In [6]:
# Checking my work
pd.read_sql("""
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders 
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;
""", conn)

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [8]:
# Your code here
pd.read_sql("""
    select products.productName, sum(orderdetails.quantityOrdered) as total_num_sold
    from products
    join orderdetails on orderdetails.productCode = products.productCode
    group by products.productCode
""", conn)

,productName,total_num_sold
0,1969 Harley Davidson Ultimate Chopper,1057
1,1952 Alpine Renault 1300,961
2,1996 Moto Guzzi 1100i,999
3,2003 Harley-Davidson Eagle Drag Bike,985
4,1972 Alfa Romeo GTA,1030
...,...,...
104,The Titanic,952
105,The Queen Mary,896
106,American Airlines: MD-11S,1085
107,Boeing X-32A JSF,960


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [13]:
# Your code here
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT

pd.read_sql("""
    select products.productName, count(distinct customers.customerNumber) as numPeopleWhoOrderedProduct from products 
    join orderdetails on products.productCode = orderdetails.productCode
    join orders on orders.orderNumber = orderdetails.orderNumber
    join customers on customers.customerNumber = orders.customerNumber
    group by products.productName
    order by numPeopleWhoOrderedProduct desc
""", conn)


,productName,numPeopleWhoOrderedProduct
0,1992 Ferrari 360 Spider red,40
1,Boeing X-32A JSF,27
2,1972 Alfa Romeo GTA,27
3,1952 Alpine Renault 1300,27
4,1934 Ford V8 Coupe,27
...,...,...
104,1958 Chevy Corvette Limited Edition,19
105,2002 Chevy Corvette,18
106,1969 Chevrolet Camaro Z28,18
107,1952 Citroen-15CV,18


## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products That Have Been Ordered by Fewer Than 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [16]:
# Your code here
pd.read_sql("""
    select employees.employeeNumber, employees.firstName, employees.lastName, offices.city, employees.officeCode
    from employees join offices on employees.officeCode = offices.officeCode
    where employeeNumber in 
    (
        select customers.salesRepEmployeeNumber as employeeNumber
        from products 
        join orderdetails on products.productCode = orderdetails.productCode
        join orders on orders.orderNumber = orderdetails.orderNumber
        join customers on customers.customerNumber = orders.customerNumber
        group by products.productName
        having count(distinct customers.customerNumber) < 20
    )
""", conn)

,employeeNumber,firstName,lastName,city,officeCode
0,1370,Gerard,Hernandez,Paris,4
1,1501,Larry,Bott,London,7


## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit Over 15K

In [22]:
# Your code here
pd.read_sql("""
    select employeeNumber, employees.firstName, employees.lastname, count(customers.customerNumber)
    from employees join customers on customers.salesRepEmployeeNumber = employees.employeeNumber
    where employeeNumber in (
        select customers.salesRepEmployeeNumber as employeeNumber from customers
        group by customers.salesRepEmployeeNumber
        having avg(customers.creditLimit) > 15000
    
    )
""", conn)

,employeeNumber,firstName,lastName,count(customers.customerNumber)
0,1165,Leslie,Jennings,100


In [20]:
pd.read_sql("""
    select customers.salesRepEmployeeNumber, avg(customers.creditLimit) from customers
        group by customers.salesRepEmployeeNumber
        having avg(customers.creditLimit) > 15000
""", conn)

,salesRepEmployeeNumber,avg(customers.creditLimit)
0,1165,100433.333333
1,1166,65266.666667
2,1188,73916.666667
3,1216,81533.333333
4,1286,66614.285714
5,1323,80887.500000
6,1337,86233.333333
7,1370,91785.714286
8,1401,81340.000000
9,1501,91187.500000


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!